In [ ]:
import requests
from bs4 import BeautifulSoup

# URL do site de nutrição esportiva
url = 'https://www.drogariasminasmais.com.br/nutricao-esportiva'

# Realizando o pedido da página
response = requests.get(url)

soup = BeautifulSoup(response.content, 'html.parser')

product_name_elements = soup.find_all('span', class_='vtex-product-summary-2-x-productBrand')

price_elements = soup.find_all('span', class_='vtex-product-price-1-x-sellingPriceValue')

if len(product_name_elements) == len(price_elements):
    for name, price in zip(product_name_elements, price_elements):
        # Extraindo o nome e o preço
        product_name = name.get_text(strip=True)
        price_value = price.get_text(strip=True)
        
        # Imprimindo o nome do produto e o preço
        print(f'Produto: {product_name} - Preço: {price_value}')
else:
    print("Número de produtos e preços não coincidem.")


In [1]:
import requests
from bs4 import BeautifulSoup

url = 'https://www.drogariasminasmais.com.br/medicamentos'

# Realizando o pedido da página
response = requests.get(url)

# Criando o objeto BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

product_name_elements = soup.find_all('span', class_='vtex-product-summary-2-x-productBrand')

price_elements = soup.find_all('span', class_='vtex-product-price-1-x-sellingPriceValue')

if len(product_name_elements) == len(price_elements):
    for name, price in zip(product_name_elements, price_elements):
        
        product_name = name.get_text(strip=True)
        price_value = price.get_text(strip=True)
        

        print(f'Produto: {product_name} - Preço: {price_value}')
else:
    print("Número de produtos e preços não coincidem.")


Produto: Centrum Pó Energia Sachê Sabor Guaraná - 5g - Preço: R$3,99
Produto: Pantoprazol 20mg com 28 Comprimidos - Cimed Genérico - Preço: R$6,14
Produto: Pantoprazol 20mg com 28 Comprimidos - Biosintetica Genérico - Preço: R$4,14
Produto: Desogestrel 0,75mg com 84 Comprimidos - Eurofarma Genérico - Preço: R$28,19
Produto: Desogestrel 0,75mg com 84 Comprimidos - Germed Genérico - Preço: R$24,46
Produto: Adeforte com 30 Cápsulas  - Gross - Preço: R$42,49
Produto: Font D 2.000UI Gotas com 10ml - União Química - Preço: R$17,66
Produto: Peagesic 300mg com 30 Comprimidos - Eurofarma - Preço: R$209,49
Produto: Dorflex com 36 Comprimidos - Sanofi - Preço: R$16,90
Produto: Dipirona Sódica Sabor Abacaxi Gotasl  20ml - Neo Química Genérico - Preço: R$5,49
Produto: Vitamina D3 7.000UI com 30 Cápsulas - Neo Química Genérico - Preço: R$10,00
Produto: Aerodini Spray com 200 Doses - Teuto - Preço: R$26,53
Produto: Colírio Teuto - 20ml - Preço: R$9,99
Produto: Leite de Magnésia Magmilk Tradicional co